# Lotka Volterra

This notebook is devoted to the understanding of the PINN applied to the Lotka Volterra problem. First let's formulate the problem. The equation that describe the problem are the following:

$$
\frac{dx}{dt} = \alpha x - \beta x y
$$

$$
\frac{dy}{dt} = \delta x y - \gamma y
$$

Let's suppose to fix the value of the parameters to:
$$
\alpha = 0.25 \\
\beta = 0.0 \\
\gamma = 2 \\
\delta = 0.25 \\
$$


Following what we did in the first_example notebook, let's first define the Neural_net. The strcture is really similar to the one of the first example, with the only addition of some layers, in order to make the network deeper. 

In [ ]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt

import torch
from torch.autograd import grad
import torch.nn as nn
from numpy import genfromtxt
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F

class Neural_net(torch.nn.Module):
    def __init__(self, n_in = 1, n_out =1):
        super(Neural_net, self).__init__()

        self.tanh = torch.nn.Tanh()

        self.layer1 = torch.nn.Linear(n_in,20)
        self.layer2 = torch.nn.Linear(20,20)
        self.layer3 = torch.nn.Linear(20,20)
        self.layer_out = torch.nn.Linear(20,n_out)

    def forward(self, x):
        x = self.layer1(x)
        x = self.tanh(x)
        x = self.layer2(x)
        x = self.tanh(x)
        x = self.layer3(x)
        x = self.tanh(x)
        x = self.layer_out(x)

        return x

As befone, let's create a class of the PINN model, where we define the wrap_grad, used to computer the derivative of `f` with respect to `x`. 

In [ ]:
class predprey_pinn():
    def wrap_grad(self, f,x):
        return torch.autograd.grad(f,x,
        grad_outputs=torch.ones_like(x),
        retain_graph=True,
        create_graph=True)[0]
 

Let's define the normalization data. The normalization is done in a really simple way. Given the un-normalized tensor `unnormed`, the minimum of of all values is subtracted. Then, the resulting value is divided by the difference of the maximum and minimum value, that is like a range. This is used to make the value more consistent as well as the neural network training. 

On the other side, the `un_normalize` function is used to un-normalize the data. It does exactly the opposite process.

In [ ]:
class predprey_pinn(predprey_pinn):
    def normalize(self, id, unnormed):
        return (unnormed - self.mins[id])/(self.maxes[id]- self.mins[id])

    def un_normalize(self, id, normed):
        return normed*(self.maxes[id] -self.mins[id])+ self.mins[id]

Let's now define the loss

In [ ]:
class predprey_pinn(predprey_pinn):
    def de_loss(self):
        pred = self.model(self.domain)
        x,y = (d.reshape(-1,1) for d in torch.unbind(pred, dim =1))
        
        dx = self.wrap_grad(x, self.domain)
        dy = self.wrap_grad(y, self.domain)

        x = self.un_normalize(0,x)
        y = self.un_normalize(1,y)

        ls0 = torch.mean((dx - (self.alpha*x -self.beta*x*y)/(self.maxes[0]-self.mins[0]) )**2)
        ls1 = torch.mean((dy -(self.delta*x*y - y*self.gamma)/(self.maxes[1]-self.mins[1]))**2)
        ic = torch.mean((self.c0-pred[0])**2)
        
        return ls0 + ls1 + ic
    
    def data_loss(self):
        x,y = torch.unbind(self.model(self.t_dat), dim = 1)
        z1 = torch.mean((x - self.x_norm)**2)
        z2 = torch.mean((y- self.y_norm)**2)
        return z1 + z2
